# Copy vs View in NumPy
- **Assignment (`=`)** → Creates a **reference** (view) to original data  
- **`.copy()`** → Creates a **duplicate** with new memory allocation  

Key difference:  
- Modifying a **view** affects original  
- Modifying a **copy** leaves original unchanged  

In [1]:
import numpy as np

# Original array
original = np.array([1, 2, 3, 4])

# Assignment creates VIEW (reference)
view_ref = original

# Modify view
view_ref[0] = 99

print("Original after view modification:", original)  # [99, 2, 3, 4]
print("Is same object?", original is view_ref)        # True

Original after view modification: [99  2  3  4]
Is same object? True


### Slicing = Implicit View  
Most slicing operations return **views** (not copies):
```python
slice_view = original[1:3]  # Shares memory with original

In [14]:

#Cell 4: Code - Slicing View Example**

arr = np.array([10, 20, 30, 40])
slice_view = arr[1:3]  # View of [20,30]

# Modify slice
slice_view[0] = 99

print("Original after slice mod:", arr)  # [10,99,30,40]
print("Base of slice:", slice_view.base is arr)  # True
print(np.shares_memory(arr,slice_view))

Original after slice mod: [10 99 30 40]
Base of slice: True
True


In [6]:
# Create original
original = np.array([1, 2, 3, 4])

# Make full copy
true_copy = original.copy()

# Modify copy
true_copy[0] = 99

print("Original after copy mod:", original)  # [1,2,3,4] (unchanged)
print("Copy:", true_copy)                   # [99,2,3,4]
print("Same memory?", np.shares_memory(original, true_copy))  # False

Original after copy mod: [1 2 3 4]
Copy: [99  2  3  4]
Same memory? False


## Operations That Create VIEWS
1. Basic slicing: `arr[1:5]`  
2. Transpose: `arr.T`  
3. Reshape: `arr.reshape(...)`  
4. Array slicing: `arr[:, 1]`  

## Operations That Create COPIES
1. Explicit: `arr.copy()`  
2. Fancy indexing: `arr[[0,2,4]]`  
3. Boolean indexing: `arr[arr > 5]`  

In [12]:
def test_memory(arr1, arr2, name):
    print(f"\n{name}:")
    print("Same object?", arr1 is arr2)
    print("Shares memory?", np.shares_memory(arr1, arr2))
    print("Same data?", np.array_equal(arr1, arr2))

# Test cases
arr = np.arange(10)

# View examples
slice_view = arr[2:5]
reshape_view = arr.reshape(2,5)
print(reshape_view)
test_memory(arr, slice_view, "Slice View")

# Copy examples
fancy_copy = arr[[0,2,4]]
bool_copy = arr[arr > 5]
test_memory(arr, fancy_copy, "Fancy Copy")

[[0 1 2 3 4]
 [5 6 7 8 9]]

Slice View:
Same object? False
Shares memory? True
Same data? False

Fancy Copy:
Same object? False
Shares memory? False
Same data? False


### Practical Implications
1. **Memory Efficiency**: Views avoid data duplication  
2. **Accidental Modification**: Changing views alters originals  
3. **Performance**: Copies have allocation overhead  

Bug Example:  
```python
data = np.array([1,2,3,4])
backup = data  # Oops! Should be data.copy()
data[0] = 99   # Also changes "backup"

In [9]:

###Cell 9: Code - When to Use Copy**

# Scenario 1: Preserve original data
original = np.random.rand(1000)
processed = original * 2  # Creates NEW array (automatic copy)
processed[0] = 0          # Safe, original unchanged

# Scenario 2: Break dependency
matrix = np.arange(1,10).reshape(3,3)
slice_view = matrix[:2, :2]  # View of top-left
slice_copy = matrix[:2, :2].copy()  # Independent copy

slice_view[0,0] = 99  # Affects matrix!
slice_copy[0,0] = 100 # Doesn't affect matrix

print("Original after operations:\n", matrix)

Original after operations:
 [[99  2  3]
 [ 4  5  6]
 [ 7  8  9]]


## How to Verify
1. `arr.base`: Returns None if owns data (copy), else reference  
2. `np.shares_memory(a, b)`: Checks memory overlap  
3. `id(arr)`: Compare object IDs (not foolproof)  

In [10]:
arr = np.array([1,2,3])

# View checks
slice_view = arr[1:]
print("\nSlice view base:", slice_view.base is arr)  # True
print("Memory shared?", np.shares_memory(arr, slice_view))  # True

# Copy checks
full_copy = arr.copy()
print("\nCopy base:", full_copy.base is None)  # True (owns data)
print("Memory shared?", np.shares_memory(arr, full_copy))  # False


Slice view base: True
Memory shared? True

Copy base: True
Memory shared? False


In [11]:
a = np.array([1,2,3])
b = a[:]  # Creates view, not copy!
b[0] = 99
print(a[0])  # What happens?

99
